In [2]:
from scipy.stats import iqr,skew,kurtosis
from datetime import datetime
from copy import deepcopy
import math
from scipy.stats import pearsonr
from sklearn.externals.joblib import Parallel,delayed
import warnings
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return average, math.sqrt(variance)

def get_rr_features(a):
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])


def get_weighted_rr_features(a):
    a = np.repeat(a[:,0],np.int64(np.round(100*a[:,1])))
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])


def get_quality_features(a):
    feature = [np.percentile(a,50),np.mean(a),
               len(a[a>.2])/60,len(a[a>.6])/60]
    return np.array(feature)

def get_daywise(data):
    return [a for i,a in data.groupby(['user','day'],as_index=False) if a[['likelihood_max_array','rr_array']].dropna().shape[0]>60]

def parse_day_data(data_day):
    data_day['likelihood_max_array'] = data_day['likelihood_max_array'].apply(lambda a:np.squeeze(a).reshape(-1,3))
    data_day['likelihood'] = data_day['likelihood_max_array'].apply(lambda a:np.max(a,axis=1))
    data_day['likelihood_ind'] = data_day['likelihood_max_array'].apply(lambda a:np.argmax(a,axis=1))
    data_day['rr_array'] = data_day['rr_array'].apply(lambda a:np.squeeze(a).reshape(-1,3))
    data_day['length'] = data_day['rr_array'].apply(lambda a:a.shape[0])
    data_day = data_day[data_day.length>20]
    data_day['time'] = data_day['ltime'].apply(lambda a:datetime.timestamp(a))
    indexes = data_day['likelihood_ind'].values
    rr_arrays = data_day['rr_array'].values
    rrs = []
    for i,rr in enumerate(rr_arrays):
        index = indexes[i]
        frr = np.squeeze(np.array([rr[i,index[i]] for i in range(rr.shape[0])]))
        rrs.append(frr)
    data_day['rr'] = rrs
    data_day['rr_col'] = data_day.apply(lambda a: np.vstack([np.squeeze(a['rr']),np.squeeze(a['likelihood']),np.squeeze(a['activity'])]).T,
                     axis=1)
    return data_day

def remove_3sd(heart_rate_window):
    temp = deepcopy(heart_rate_window)
    try:
        r,tt = weighted_avg_and_std(heart_rate_window[heart_rate_window[:,1]>.25,0],heart_rate_window[heart_rate_window[:,1]>.25,1])
        index = np.where((heart_rate_window[:,0]<r+3*tt)&(heart_rate_window[:,0]>r-3*tt))[0]
        heart_rate_window = heart_rate_window[index]
    except:
        pass
    if heart_rate_window.shape[0]>10:
        return [heart_rate_window,'Available']
    else:
        return [temp[:10],'Not Available']

    
def parse_for_features(data_day):
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:a[np.where((a[:,1]>.05)&(a[:,0]>300)&(a[:,0]<1500)&(a[:,2]<.2))[0],:2])
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:remove_3sd(a))
    data_day['length1'] = data_day['rr_col'].apply(lambda a:a[0].shape[0])
    data_day = data_day[data_day.length1>30]
    data_day['indicator'] = data_day['rr_col'].apply(lambda a:a[1])
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:a[0])
    data_day['likelihood'] = data_day['rr_col'].apply(lambda a:a[:,1])
    data_day['rr'] = data_day['rr_col'].apply(lambda a:a[:,0])
    data_day['rr_col1'] = data_day.apply(lambda a:np.vstack([list(a['rr']),list(a['likelihood'])]).T,axis=1)
    data_day['rr_features'] = data_day['rr'].apply(lambda a:get_rr_features(a))
    data_day['rr_weighted_features'] = data_day['rr_col1'].apply(lambda a:get_weighted_rr_features(a))
    data_day['quality_features'] = data_day['likelihood'].apply(lambda a:get_quality_features(a))
    data_day['quality_mag'] = data_day['quality_features'].apply(lambda a:np.sum(a)/len(a))
    return data_day

def normalize_daywise(feature_matrix,quals1):
    for i in range(feature_matrix.shape[1]):
        m,s = weighted_avg_and_std(feature_matrix[:,i], quals1)
        feature_matrix[:,i]  = (feature_matrix[:,i] - m)/s
    return feature_matrix

def smooth(y, box_pts=10):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def parse_day_data_ecg(data_day):
    data_day = data_day[['ecg_rr_array','ltime','window']].dropna()
    data_day['count_ecg'] = data_day['ecg_rr_array'].apply(lambda a:len(a))
    data_day = data_day[data_day.count_ecg>20]
    data_day['ecg_rr_array_final'] = data_day['ecg_rr_array']
    data_day['ecg_features'] = data_day['ecg_rr_array_final'].apply(lambda a:get_rr_features(smooth(a)))
    return data_day

def parse_each_day_ppg_ecg(a):
    columns = ['window', 'ltime', 'likelihood_max_array', 'activity', 'rr_array',
       'time', 'timestamp', 'likelihood_mean', 'localtime', 'ecg_rr_array',
       'day', 'version', 'user', 'quality_features', 'activity_features', 'likelihood', 'likelihood_ind', 'length', 'rr', 'rr_col',
       'length1', 'indicator', 'rr_col1', 'rr_features',
       'rr_weighted_features', 'quality_mag', 'ecg_rr_array_final', 'ecg_features']
    ecg_columns = ['window', 'ecg_rr_array_final','ecg_features']
    a = a.drop(['stress_likelihood', 'stress_likelihood_ecg'],axis=1)
    a_ecg = pd.DataFrame([],columns=ecg_columns)
    if a['ecg_rr_array'].dropna().shape[0]>60:
        a_ecg = parse_day_data_ecg(deepcopy(a))
        a_ecg = a_ecg[ecg_columns]
    a_ppg = parse_day_data(a)
    a_ppg = parse_for_features(a_ppg)
    if a_ppg.shape[0]==0:
        return pd.DataFrame([],columns=columns)
    a_ppg = pd.merge(a_ppg, a_ecg, how='left',left_on=['window'],right_on=['window'])
    if a_ppg.shape[0]<60:
        return pd.DataFrame([],columns=columns)
    print(a_ppg.shape)
    a_ppg = get_ecg_stress(a_ppg)
    print(a_ppg.shape)
    a_ppg = get_ppg_stress(a_ppg)
    print(a_ppg.shape)
    return a_ppg

def get_ppg_stress(a):
    clf = pickle.load(open('../models/stress_ecg_final.p','rb'))
    quals1 = np.array(list(a['quality_mag'].values))
    feature_matrix = np.array(list(a['rr_weighted_features']))
    if len(feature_matrix)<10:
        a['stress_likelihood_ppg'] = np.nan
        return a
    rr_70th = np.percentile(feature_matrix[:,2],60)
    rr_95th = np.percentile(feature_matrix[:,2],99)
    index = np.where((feature_matrix[:,2]>rr_70th)&(feature_matrix[:,2]<rr_95th))[0]
    for i in range(feature_matrix.shape[1]):
        m,s = weighted_avg_and_std(feature_matrix[index,i], quals1[index])
        feature_matrix[:,i]  = (feature_matrix[:,i] - m)/s
    probs = clf.predict_proba(feature_matrix)[:,1]
    a['stress_likelihood_ppg'] = probs
    return a

def get_ecg_stress(a):
    clf = pickle.load(open('../models/stress_ecg_final.p','rb'))
    a_ecg = deepcopy(a[['window','ecg_features']].dropna())
    feature_matrix = np.array(list(a_ecg['ecg_features']))
    if len(feature_matrix)<10:
        a['stress_likelihood_ecg'] = np.nan
        return a
    rr_70th = np.percentile(feature_matrix[:,2],60)
    rr_95th = np.percentile(feature_matrix[:,2],99)
    index = np.where((feature_matrix[:,2]>rr_70th)&(feature_matrix[:,2]<rr_95th))[0]
    means = np.mean(feature_matrix[index],axis=0)
    stds = np.std(feature_matrix[index],axis=0)
    feature_matrix = (feature_matrix - means)/stds
    probs = clf.predict_proba(feature_matrix)[:,1]
    a_ecg['stress_likelihood_ecg'] = probs
    a_ecg = a_ecg.drop(['ecg_features'],axis=1)
    a = pd.merge(a, a_ecg, how='left', left_on=['window'], right_on=['window'])
    return a
    
def parse_each_participant(directory,d):
    data = pickle.load(open(directory+d,'rb')).reset_index(drop=True)
    print(data.shape,d)
    ema = data[['user','day','window','time','ltime','all_scores','score','label']]
    data = data.drop(['all_scores','score','label'],axis=1)
    data_all = get_daywise(data)
    if len(data_all)==0:
        return 0
    final_output = Parallel(n_jobs=25,verbose=4)(delayed(parse_each_day_ppg_ecg)(a) for a in data_all)
#     final_output = [parse_each_day_ppg_ecg(a) for a in data_all]
    final_output = [a for a in final_output if a.shape[0]>0]
    if len(final_output)==0:
        return 0
    final_output = pd.concat(final_output)
    final_output['stress_likelihood_ppg_qual'] = final_output['stress_likelihood_ppg']
    print(final_output.shape)
#     print(final_output.shape,final_output.columns)
    pickle.dump([final_output,ema],open(directory1+d,'wb'))
    return 0

directory = '../../cc3/rice_data/ecg_ppg_25_left3/'
directory1 = '../../cc3/rice_data/ecg_ppg_25_left5/'
# all_data = Parallel(n_jobs=30,verbose=2)(delayed(parse_each_participant)(directory,d) for d in os.listdir(directory)[:2] if d[-1]=='p')
all_data = [parse_each_participant(directory,d) for d in os.listdir(directory) if d=='4bf6078d-afcd-432a-a8a5-8b5e8a4eda9e.p']

(10883, 20) 4bf6078d-afcd-432a-a8a5-8b5e8a4eda9e.p


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  14 | elapsed:    4.3s remaining:    7.8s
[Parallel(n_jobs=25)]: Done   9 out of  14 | elapsed:    4.8s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  14 out of  14 | elapsed:    6.1s finished


(9484, 31)


In [ ]:
from scipy.stats import iqr,skew,kurtosis
from datetime import datetime
from copy import deepcopy
import math
from scipy.stats import pearsonr
from sklearn.externals.joblib import Parallel,delayed
import warnings
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return average, math.sqrt(variance)

def get_rr_features(a):
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])


def get_weighted_rr_features(a):
    a = np.repeat(a[:,0],np.int64(np.round(100*a[:,1])))
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])


def get_quality_features(a):
    feature = [np.percentile(a,50),np.mean(a),
               len(a[a>.2])/60,len(a[a>.6])/60]
    return np.array(feature)

def get_daywise(data):
    return [a for i,a in data.groupby(['user','day'],as_index=False) if a[['likelihood_max_array','rr_array']].dropna().shape[0]>60]

def parse_day_data(data_day):
    data_day['likelihood_max_array'] = data_day['likelihood_max_array'].apply(lambda a:np.squeeze(a).reshape(-1,3))
    data_day['likelihood'] = data_day['likelihood_max_array'].apply(lambda a:np.max(a,axis=1))
    data_day['likelihood_ind'] = data_day['likelihood_max_array'].apply(lambda a:np.argmax(a,axis=1))
    data_day['rr_array'] = data_day['rr_array'].apply(lambda a:np.squeeze(a).reshape(-1,3))
    data_day['length'] = data_day['rr_array'].apply(lambda a:a.shape[0])
    data_day = data_day[data_day.length>20]
    data_day['time'] = data_day['ltime'].apply(lambda a:datetime.timestamp(a))
    indexes = data_day['likelihood_ind'].values
    rr_arrays = data_day['rr_array'].values
    rrs = []
    for i,rr in enumerate(rr_arrays):
        index = indexes[i]
        frr = np.squeeze(np.array([rr[i,index[i]] for i in range(rr.shape[0])]))
        rrs.append(frr)
    data_day['rr'] = rrs
    data_day['rr_col'] = data_day.apply(lambda a: np.vstack([np.squeeze(a['rr']),np.squeeze(a['likelihood']),np.squeeze(a['activity'])]).T,
                     axis=1)
    return data_day

def remove_3sd(heart_rate_window):
    temp = deepcopy(heart_rate_window)
    try:
        r,tt = weighted_avg_and_std(heart_rate_window[heart_rate_window[:,1]>.25,0],heart_rate_window[heart_rate_window[:,1]>.25,1])
        index = np.where((heart_rate_window[:,0]<r+3*tt)&(heart_rate_window[:,0]>r-3*tt))[0]
        heart_rate_window = heart_rate_window[index]
    except:
        pass
    if heart_rate_window.shape[0]>10:
        return [heart_rate_window,'Available']
    else:
        return [temp[:10],'Not Available']

    
def parse_for_features(data_day):
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:a[np.where((a[:,1]>.05)&(a[:,0]>300)&(a[:,0]<1500)&(a[:,2]<.2))[0],:2])
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:remove_3sd(a))
    data_day['length1'] = data_day['rr_col'].apply(lambda a:a[0].shape[0])
    data_day = data_day[data_day.length1>30]
    data_day['indicator'] = data_day['rr_col'].apply(lambda a:a[1])
    data_day['rr_col'] = data_day['rr_col'].apply(lambda a:a[0])
    data_day['likelihood'] = data_day['rr_col'].apply(lambda a:a[:,1])
    data_day['rr'] = data_day['rr_col'].apply(lambda a:a[:,0])
    data_day['rr_col1'] = data_day.apply(lambda a:np.vstack([list(a['rr']),list(a['likelihood'])]).T,axis=1)
    data_day['rr_features'] = data_day['rr'].apply(lambda a:get_rr_features(a))
    data_day['rr_weighted_features'] = data_day['rr_col1'].apply(lambda a:get_weighted_rr_features(a))
    data_day['quality_features'] = data_day['likelihood'].apply(lambda a:get_quality_features(a))
    data_day['quality_mag'] = data_day['quality_features'].apply(lambda a:np.sum(a)/len(a))
    return data_day

def normalize_daywise(feature_matrix,quals1):
    for i in range(feature_matrix.shape[1]):
        m,s = weighted_avg_and_std(feature_matrix[:,i], quals1)
        feature_matrix[:,i]  = (feature_matrix[:,i] - m)/s
    return feature_matrix


def parse_day_data_ecg(data_day):
    data_day = data_day[['ecg_rr_array','ltime','window']].dropna()
    data_day['count_ecg'] = data_day['ecg_rr_array'].apply(lambda a:len(a))
    data_day = data_day[data_day.count_ecg>20]
    data_day['ecg_rr_array_final'] = data_day['ecg_rr_array']
    data_day['ecg_features'] = data_day['ecg_rr_array_final'].apply(lambda a:get_rr_features(a))
    return data_day

def parse_each_day_ppg_ecg(a):
    columns = ['window', 'ltime', 'likelihood_max_array', 'activity', 'rr_array',
       'time', 'timestamp', 'likelihood_mean', 'localtime', 'ecg_rr_array',
       'day', 'version', 'user', 'quality_features', 'activity_features', 'likelihood', 'likelihood_ind', 'length', 'rr', 'rr_col',
       'length1', 'indicator', 'rr_col1', 'rr_features',
       'rr_weighted_features', 'quality_mag','window', 'ecg_rr_array_final', 'ecg_features']
    ecg_columns = ['window', 'ecg_rr_array_final','ecg_features']
    a = a.drop(['stress_likelihood', 'stress_likelihood_ecg'],axis=1)
    a_ecg = pd.DataFrame([],columns=ecg_columns)
    if a['ecg_rr_array'].dropna().shape[0]>60:
        a_ecg = parse_day_data_ecg(deepcopy(a))
        a_ecg = a_ecg[ecg_columns]
    a_ppg = parse_day_data(a)
    a_ppg = parse_for_features(a_ppg)
    if a_ppg.shape[0]==0:
        return pd.DataFrame([],columns=columns)
#     a_ecg = a_ecg.rename({'window': 'window1'}, axis=1)
    a_ppg = pd.merge(a_ppg, a_ecg, how='left', left_on=['window'],right_on=['window'],suffixes=('', '_7'))
#     print(a_ppg.columns,'r')
    return a_ppg[columns]

def get_both_stress(a):
    clf = pickle.load(open('../models/stress_ecg_final.p','rb'))
    a_ecg = deepcopy(a[['window','ecg_features']].dropna())
    print(a_ecg.columns,'--'*20)
    feature_matrix = np.array(list(a_ecg['ecg_features']))
    rr_70th = np.percentile(feature_matrix[:,2],60)
    rr_95th = np.percentile(feature_matrix[:,2],99)
    index = np.where((feature_matrix[:,2]>rr_70th)&(feature_matrix[:,2]<rr_95th))[0]
    means = np.mean(feature_matrix[index],axis=0)
    stds = np.std(feature_matrix[index],axis=0)
    feature_matrix = (feature_matrix - means)/stds
    probs = clf.predict_proba(feature_matrix)[:,1]
    a_ecg['stress_likelihood_ecg'] = probs
    a = a.drop(['ecg_features'],axis=1)
#     print(a.columns,a_ecg.columns)
#     a_ecg = a_ecg.rename({'window': 'window1'}, axis=1)
    print(a_ecg.columns)
    a = pd.merge(a, a_ecg, how='left', on=['window'])
#     a = a.drop(['window1'],axis=1)
#     plt.figure(figsize=(16,10))
#     plt.plot(a['timestamp'],a['stress_likelihood_ecg'])
#     plt.show()
    return a
    
def parse_each_participant(directory,d):
    data = pickle.load(open(directory+d,'rb')).reset_index(drop=True)
    ema = data[['user','day','window','time','ltime','all_scores','score','label']]
    data = data.drop(['all_scores','score','label'],axis=1)
    data_all = get_daywise(data)
    if len(data_all)==0:
        return 0
    final_output = Parallel(n_jobs=25,verbose=4)(delayed(parse_each_day_ppg_ecg)(a) for a in data_all)
#     final_output = [parse_each_day_ppg_ecg(a) for a in data_all]
    final_output = [a for a in final_output if a.shape[0]>0]
    if len(final_output)==0:
        return 0
    final_output = pd.concat(final_output)
    print(final_output.columns)
    final_output = get_both_stress(final_output)
#     pickle.dump([final_output,ema],open(directory1+d,'wb'))
    return 0
directory = '../../cc3/rice_data/ecg_ppg_25_left3/'
directory1 = '../../cc3/rice_data/ecg_ppg_25_left5/'
# all_data = Parallel(n_jobs=30,verbose=2)(delayed(parse_each_participant)(directory,d) for d in os.listdir(directory)[:2] if d[-1]=='p')
all_data = [parse_each_participant(directory,d) for d in os.listdir(directory) if d[-1]=='p']

In [ ]:
data1 = np.concatenate([a[0] for a in all_data])
yld = np.concatenate([a[1] for a in all_data])
yld1 = yld[:,:2]
yld = yld[:,2:]
day_corr = np.concatenate([a[2] for a in all_data])

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':25})
plt.figure(figsize=(16,8))
plt.boxplot(yld)
plt.ylabel('Minutes')
plt.xticks(range(1,yld.shape[1]+1),['ECG YIELD','PPG YIELD'])
plt.title('Stress yield across all participant days')
plt.show()

In [ ]:
print(day_corr.shape)
day_corr = day_corr[~np.isnan(day_corr).any(axis=1)]
print(np.sum([a[4] for a in all_data]),'- Participant Days,',np.sum([a[3] for a in all_data]),'- Users')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})
plt.figure(figsize=(16,8))
plt.boxplot(day_corr[~np.isnan(day_corr).any(axis=1)][:,np.array([0,1,2])])
plt.ylim([-1,1])
plt.ylabel('Pearson Correlation')
plt.xticks(range(1,day_corr.shape[1]+1),['Original cStress','cStress with Weighted Features and weighted normalization','cStress with Weighted Features'],rotation=10)
plt.title('Correlation with ECG For Different Modes of Normalization')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})
plt.figure(figsize=(16,8))
plt.boxplot(day_corr[~np.isnan(day_corr).any(axis=1)][:,np.array([0,1,2])])
plt.ylabel('Pearson Correlation')
plt.xticks(range(1,day_corr.shape[1]+1),['Original cStress','cStress with Weighted Features and weighted normalization','cStress with Weighted Features'],rotation=10)
plt.title('Correlation with ECG For Different Modes of Normalization')
plt.show()

In [ ]:
data_all = pd.DataFrame(data1,columns=['quality','corr_orig','corr_new','corr_new1','corr_between','ppg_yield','ecg_yield'])
data_all1 = pd.DataFrame(yld1,columns=['quality','ppg_yield'])

corr_25 = data_all.groupby('quality').quantile(.5)
x = corr_25.index.values
x1 = np.unique(data_all1['quality'].values)
y = []
for a in x1:
    y.append(data_all1[data_all1.quality>=a]['ppg_yield'].sum()/60/np.sum([a[4] for a in all_data]))

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size':20})
fig, ax1 = plt.subplots(figsize=(20,12))
ax2 = ax1.twinx()
ax1.plot(x,corr_25['corr_orig'].loc[x],label='Original Cstress')
ax1.plot(x,corr_25['corr_new'].loc[x],label='Weighted Normalization with weighted features')
ax1.plot(x,corr_25['corr_new1'].loc[x],label='Original Cstress using Weighted Features')
# ax1.plot(x,corr_25['corr_between'].loc[x],label='Original Normalization using auto Features')
ax2.plot(x1,y,label='PPG Yield')
ax1.grid()
# ax1.plot(x,corr_75['corr_orig'].loc[x],label='Original 75th')
# ax1.plot(x,corr_75['corr_new'].loc[x],label='Weighted 75th')
ax1.legend(fontsize=20)
ax1.set_xlabel('Quality Metric')
ax2.set_ylabel('Median Hours per Participant Day', color='g')
ax1.set_ylabel('Median Correlation Across all Participant Days', color='b')
plt.show()
#  plt.figure(figsize=(16,8))

In [ ]:
data_all = pd.DataFrame(data1,columns=['quality','corr_orig','corr_new','corr_new1','corr_between','ppg_yield','ecg_yield'])
data_all1 = pd.DataFrame(yld1,columns=['quality','ppg_yield'])

corr_25 = data_all.groupby('quality').quantile(.5)
x = corr_25.index.values
x1 = np.unique(data_all1['quality'].values)
y = []
for a in x1:
    y.append(data_all1[data_all1.quality>=a]['ppg_yield'].sum()/60/np.sum([a[4] for a in all_data]))

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size':20})
fig, ax1 = plt.subplots(figsize=(20,12))
ax2 = ax1.twinx()
ax1.plot(x,corr_25['corr_orig'].loc[x],label='Original Cstress')
ax1.plot(x,corr_25['corr_new'].loc[x],label='Weighted Normalization with weighted features')
ax1.plot(x,corr_25['corr_new1'].loc[x],label='Original Cstress using Weighted Features')
# ax1.plot(x,corr_25['corr_between'].loc[x],label='Original Normalization using auto Features')
ax2.plot(x1,y,label='PPG Yield')
ax1.grid()
# ax1.plot(x,corr_75['corr_orig'].loc[x],label='Original 75th')
# ax1.plot(x,corr_75['corr_new'].loc[x],label='Weighted 75th')
ax1.legend(fontsize=20)
ax1.set_xlabel('Quality Metric')
ax2.set_ylabel('Median Hours per Participant Day', color='g')
ax1.set_ylabel('Median Correlation Across all Participant Days', color='b')
plt.show()


# plt.figure(figsize=(16,8))



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(16,8))
plt.suptitle('')
c = data_all.boxplot(column=['corr_new'], by='quality', ax=ax,showfliers=True)
plt.ylim([-3,1])
plt.xticks(rotation=100)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(16,8))
plt.suptitle('')
c = data_all.boxplot(column=['ppg_yield'], by='quality', ax=ax)

In [ ]:
data_all.groupby('quality').quantile([.25,.75]).loc[(0.2, 0.25)]

In [ ]:
import sklearn

In [ ]:
sklearn.show_versions()

In [ ]:
data_all1['quality']

In [ ]:
import pickle
pickle.load(open('../models/stress_model_ecg_2.p','rb'))